In [ ]:
import cv2
import holoviews as hv
from holoviews import opts
import param
import panel as pn
import numpy as np
import holoviews as hv
from holoviews import opts
import numpy as np
import pandas as pd
pn.extension()
hv.extension('bokeh')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# CONSTANTS
VISUAL_WIDTH = 800
VISUAL_HEIGHT = 800

def visualize(graph):
    return graph.opts(width=VISUAL_WIDTH, height=VISUAL_HEIGHT)

# Data Loading

Use this cell to point to the location of your 3 sources
1. MRI source (this the folder path)
1. Pathology source (this is the image path)
1. Pathology annotated image source (this is the annotated image path)

In [ ]:
mri_folder_path = f'data/Prostate Fused-MRI-Pathology/aaa0051/07-02-2000-PELVISPROSTATE-97855/4.000000-T2 AXIAL SM FOV-36207'
pathology_img_path = 'data/pathology/aaa 0051 BSlice.tif'
pathology_annotated_img_path = 'data/pathology/aaa 0051 BSlice annotated.tif'

# mri_folder_path = 'data/patient87/Prostate Fused-MRI-Pathology/aaa0087/09-01-2000-PELVISPROSTATE-81724/5.000000-T2 AXIAL SM FOV-30095'
# pathology_img_path = '/Users/neshdev/radpathfusion/data/patient87/pathology/aaa0087 A1A2A3A4.tif'
# pathology_annotated_img_path


**Warning**: This might take a while depening on the size of the image

In [ ]:
import utils as rad

In [ ]:
pathology_source = rad.Image.load(pathology_img_path)
pathology_annotated_source = rad.Image.load(pathology_annotated_img_path)

In [ ]:
# pathology_source = cv2.imread(pathology_img_path)
# pathology_annotated_source = cv2.imread(pathology_annotated_img_path)

# Select and Upscale the MRI slice

Use the follwing cell to select the slice of the MRI image. Additionally, select the upscaled ascpet ratio of the MRI image.

In [ ]:
mri_viewer = rad.MriVisualizer(mri_path=mri_folder_path)
mri_viewer.panel()

# Downscale the Pathology image

In the following cell, downscale the pathology image. Similarly, there are options to transform the image.

Note: This is normally a very large image file. Downscaling this image as small as possible to find the orientation.
      
**WARNING**: The aspect ratio is of the original image at angle 0. Not the rotated bounds of the chart that are visually displayed.

In [ ]:
pathology_viewer = rad.PathologyVisualizer(pathology_img=pathology_source)
pathology_viewer.panel()

# Annotation Overlay Validation

The below visualization is to show the annotated pathology image along with the pathology image.

In [ ]:
pathology_viewer.side_by_side(pathology_annotated_source)

# Landmark Selection

After completing the upscaling and downscaling of both the mri image and pathology image. We will use the next control to select the landmarks.

Use the Point Draw tool to select the points. Refrence to point draw selection can be found here: https://holoviews.org/reference/streams/bokeh/PointDraw.html

Note: You can zoom and pan by using the controls.

**WARNING**: Select the same amount of points in both images. The points are associated by color. A new point will always be colored white upon creation and will be changed to the default color if the context changes.

**WARNING**: If you need to reselect the MRI slice or reorient the pathology image, you can go above and update the values in the above control. However, for it to take effect, you will have rerun the following cell.

In [ ]:
landmarks_viewer = rad.LandmarkSelectorVisualizer(fixed_img=mri_viewer.fixed_image, moving_img=pathology_viewer.moving_image)
landmarks_viewer.panel()

# Landmark association verification

The below cell is a table verifying the associated points selected from the pervious cell.

**Note**: The origin(reference frame) is bottom left. Depending on the library you are using, there are different conventions.

In [ ]:
landmarks_viewer.associated_points_to_DF()

In [ ]:
fixed = landmarks_viewer.fixed_image(in_origin=rad.Origin.TOP_LEFT)

In [ ]:
moving = landmarks_viewer.moving_image(in_origin=rad.Origin.TOP_LEFT)

In [ ]:
annotated = (pathology_annotated_source.copy_attributes(landmarks_viewer.moving_image(in_origin=rad.Origin.TOP_LEFT))
                                       .update(interpolation='INTER_AREA'))

In [ ]:
tps_viewer = rad.TpsAlgoVisualizer(
    fixed_image = landmarks_viewer.fixed_image(in_origin=rad.Origin.TOP_LEFT),
    moving_image = landmarks_viewer.moving_image(in_origin=rad.Origin.TOP_LEFT)
)


In [ ]:
visualize(tps_viewer.mri_with_fixed_points())

In [ ]:
visualize(tps_viewer.pathology_with_affine_points())

In [ ]:
visualize(tps_viewer.pathology_with_tps_points())

In [ ]:
visualize(tps_viewer.mri_with_affine_overlay())

In [ ]:
visualize(tps_viewer.mri_with_affine_and_tps_overlay())

In [ ]:
visualize(tps_viewer.mri_with_tps_overlay())

In [ ]:
tps_viewer.panel()

# Image Alignment Visualization

The below visualization shows the final transformations of the image registration done in the previous steps. In this visualization, you can see the pathology and mri image overlayed on top of each other. Additionally, you can control the transparency of each layer.

In [ ]:

overlay_viewer = rad.TpsOverlayVisualizer(
    fixed=landmarks_viewer.fixed_image(in_origin=rad.Origin.TOP_LEFT), 
    warped=landmarks_viewer.moving_image(in_origin=rad.Origin.TOP_LEFT), 
    annotated=pathology_annotated_source.copy_attributes(
        landmarks_viewer.moving_image(in_origin=rad.Origin.TOP_LEFT)))
overlay_viewer.panel()

In [ ]:
save_path = ""
landmarks_viewer.fixed_image(in_origin=rad.Origin.TOP_LEFT)